#### Variational inference description and implementation

In Bayesian computation we are most likely interested in making inference about hidden variables. From frequentist perspective we are interested in parameters of the population that most likely have generated the sample we observe. In Bayesian perspective, on the other hand, we are interested in what we know about possible values of parameters given the fact that we observe the data at hand.

$$
\begin{align}
P(Z|X) &= \frac{P(X|Z)P(Z)}{P(X)}
        &= \frac{P(X|Z)P(Z)}{\int_Z P(X|Z)P(Z)}
\end{align}
$$

Usually enough, we cannot compute the sum in the denominator easily. Variational inference is a method  for approximating the posterior of the hidden variables. The way we do it is we define a distribution Q with parameters $\phi$ and we try to make this variational distribution as close as possible to the actual posterior of the hidden variables by finding the optimal values of $\phi$.

The difference between two distributions is measured by KL divergence. Thus we are trying to minimize


$$
\begin{align}
KL(q(z|\phi)|p(z|x)) &= \langle \log q(z|\phi) - \log p(z|x) \rangle_{q(z|\phi)} \\
                     &= \langle \log q(z|\phi) - \log p(z, x) + \log p(x) \rangle_{q(z|\phi)} \\
                     &= \langle \log q(z|\phi) - \log p(z, x) \rangle_{q(z|\phi)} + \log p(x)  \geq 0 \\
\end{align}
$$

since KL divergence is always nonnegative. Since \log p(x) does not depend on q, we aim to maximize $\langle \log q(z|\phi) - \log p(z, x) \rangle$ or minimize its negative. This value is called evidence lower bound (ELBO) since:

$$
\begin{align}
    \log p(x)  \geq \langle - \log q(z|\phi) + \log p(z, x)  \rangle_{q(z|x)}
\end{align}
$$

Let us demonstrate this idea on a simple example.

$$
\begin{align}
\pi &\sim B(\alpha, \beta) \\
x_i &\sim BE(\pi)
\end{align}
$$

Let our variational distribution be $q(z|\phi) = q(\pi|a,b) = B(a, b)$. Then ELBO is:

$$
\begin{align}
\mathcal{L} &= \langle -(a-1)\log \pi - (b-1)\log (1- \pi) + \log B(a, b) +(\alpha -1)\log \pi + (\beta-1)\log (1-\pi) - \log B(\alpha, \beta) + \sum_i x_i \log \pi + (1 - x_i) \log (1-\pi)    \rangle_{q(\pi|a,b)} \\
            &= \langle (\alpha - a + \sum_i x_i)\log \pi + (\beta - b + \sum_i (1 - x_i))\log (1- \pi) \rangle_{q(\pi|a,b)} + \log B(a, b) - \log B(\alpha, \beta)  \\
            &= (\alpha - a + \sum_i x_i)\langle \log \pi \rangle_{q(\pi|a,b)} + (\beta - b + \sum_i (1 - x_i))\langle\log (1- \pi)\rangle_{q(\pi|a,b)}  + \log B(a, b) - \log B(\alpha, \beta)  \\
            &= (\alpha - a + \sum_i x_i)(\psi(a)-\psi(a + b)) + (\beta - b + \sum_i (1 - x_i))(\psi(b)-\psi(a + b))  + \log B(a, b) - \log B(\alpha, \beta)  \\
            &= (\alpha - a + \sum_i x_i)(\psi(a)-\psi(a + b)) + (\beta - b + \sum_i (1 - x_i))(\psi(b)-\psi(a + b))  + \log B(a, b) - \log B(\alpha, \beta)  \\
\end{align}
$$






When we take the derivative of the ELBO with respect to a and b:

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial a} &= (\alpha - a + \sum_i x_i)(\psi(a) - \psi(a + b))' - (\beta - b + \sum_i (1 - x_i))\psi'(a + b)\\
\frac{\partial \mathcal{L}}{\partial b} &= - (\alpha - a + \sum_i x_i)\psi'(a + b) - (\beta - b + \sum_i (1 - x_i))(\psi(a) - \psi(a + b))'
\end{align}
$$

Both equations equal 0 when:

$$
\begin{align}
a &= \alpha + \sum_i x_i \\
b &= \beta + \sum_i (1 - x_i)
\end{align}
$$

Thus, 
$$
q(\pi|a,b) = B(\alpha + \sum_i x_i, \beta + \sum_i (1 - x_i))
$$

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import beta

In [15]:
alpha_par = 1
beta_par = 1
b = beta(alpha_par, beta_par)
pi = b.rvs()
print("pi = {}".format(pi))

pi = 0.8364746676128703


In [16]:
def plot_beta(b):
    x = np.linspace(0,1,100)
    y = b.pdf(x)
    plt.plot(x,y)